# Assignment 2


The goal of the skip-gram word2vec algorithm is to accurately learn the probability distribution $P(O|C)$.
Given a specific word $o$ and a specific word $c$, we want to calculate $P(O = o|C = c)$, which is the probability
that word $o$ is an ‘outside’ word for $c$, i.e., the probability that $o$ falls within the contextual window of $c$.

Para esto, sabemos que el método de softmax puede proponer una ddp condicional como:
$$
P(c_i|\mathbf{x}) = \frac{\exp(w_i^T \mathbf{x})}{\sum_{j=1}^{C}\exp(w_j^T\mathbf{x})}
$$
y estimar los parámetros $w_1,\ldots,w_C$. Entonces, en este caso queda:

$$
P(u_o|v_c) = \frac{\exp(u_o^T v_c)}{\sum_{w\in V}\exp(u_w^Tv_c)}
$$

y estimamos 

$$
U = 
\begin{bmatrix}
 | & & | \\
u_1 & \ldots & u_{|V|} \\
| & & | \\
\end{bmatrix}
$$
con $|V|$ el largo del vocabulario. 

## 1. Cálculo a mano del gradiente:

Recall from lectures that, for a single pair of words c and o, the loss is given by:

$$
J(v_c,o,U) = - \log( P(O=o|C=c) )
$$

Another way to view this loss is as the cross-entropy 2 between the true distribution y and the predicted
distribution ŷ. Here, both y and ŷ are vectors with length equal to the number of words in the vocabulary.
Furthermore, the k th entry in these vectors indicates the conditional probability of the k th word being an
‘outside word’ for the given c. The true empirical distribution y is a one-hot vector with a 1 for the true out-
side word o, and 0 everywhere else. The predicted distribution ŷ is the probability distribution P (O|C = c)
given by our model in equation (1) (la definición de $P(O=o|C=c)$ ).

1. (3 points) Show that the naive-softmax loss given in Equation (2) is the same as the cross-entropy loss between y and ŷ; i.e., show that $$ - \sum_{w \in V} y_w log(\hat{y}_w) = -log(\hat{y}_o) $$ Your answer should be one line.

2. (5 points) Compute the partial derivative of J naive-softmax (v c , o, U ) with respect to v c . Please write your answer in terms of y, ŷ, and U .

3. (5 points) Compute the partial derivatives of J naive-softmax (v c , o, U ) with respect to each of the ‘outside’ word vectors, u w ’s. There will be two cases: when w = o, the true ‘outside’ word vector, and w 6 = o, for all other words. Please write you answer in terms of y, ŷ, and v c .

4. (3 Points) The sigmoid function is given by $$ \sigma(x) = \frac{1}{1+e^{-x}$$ Please compute the derivative of σ(x) with respect to x, where x is a vector.

5. (4 points) Now we shall consider the Negative Sampling loss, which is an alternative to the Naive Softmax loss. Assume that K negative samples (words) are drawn from the vocabulary. For simplicity of notation we shall refer to them as w 1 , w 2 , . . . , w K and their outside vectors as u 1 , . . . , u K . Note that o ∈ / {w 1 , . . . , w K }. For a center word c and an outside word o, the negative sampling loss function is given by: $$ J(v_c,o,U) = - \log( \sigma( u_o^T v_c) = - \sum_{k=1}^K log(\sigma(-u_k^T v_c)) $$ for a sample w 1 , . . . w K , where σ(·) is the sigmoid function. Please repeat parts (b) and (c), computing the partial derivatives of J neg-sample with respect to v c , with respect to u o , and with respect to a negative sample u k . Please write your answers in terms of the vectors u o , v c , and u k , where k ∈ [1, K]. After you’ve done this, describe with one sentence why this loss function is much more efficient to compute than the naive-softmax loss. Note, you should be able to use your solution to part (d) to help compute the necessary gradients here.

sigue... ver el pdf del assignment 2.

In [1]:
import random
import numpy as np
from utils.treebank import StanfordSentiment
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import time

from word2vec import *
from sgd import *

# Check Python Version
import sys
assert sys.version_info[0] == 3
assert sys.version_info[1] >= 5

In [2]:
# Reset the random seed to make sure that everyone gets the same results
random.seed(314)
dataset = StanfordSentiment()
tokens = dataset.tokens()
nWords = len(tokens)

In [6]:
nWords

19539